In [30]:
!apt-get install openjdk-8-jdk-headless -qq
!pip install pyspark

Selecting previously unselected package libxtst6:amd64.
(Reading database ... 122541 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u372-ga~us1-0ubuntu1~20.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u372-ga~us1-0ubuntu1~20.04) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64.
Preparing to unpack .../openjdk-8-jdk-headless_8u372-ga~us1-0ubuntu1~20.04_amd64.deb ...
Unpacking openjdk-8-jdk-headless:amd64 (8u372-ga~us1-0ubuntu1~20.04) ...
Setting up libxtst6:amd64 (2:1.2.3-1) ...
Setting up openjdk-8-jre-headless:amd64 (8u372-ga~us1-0ubuntu1~20.04) ...
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/orbd to provide /usr/bin/orbd (orbd) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/ser

In [31]:
!wget https://jdbc.postgresql.org/download/postgresql-42.6.0.jar

--2023-06-13 13:50:58--  https://jdbc.postgresql.org/download/postgresql-42.6.0.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1081604 (1.0M) [application/java-archive]
Saving to: ‘postgresql-42.6.0.jar’

postgresql-42.6.0.j 100%[===================>]   1.03M  --.-KB/s    in 0.08s   

2023-06-13 13:50:58 (12.3 MB/s) - ‘postgresql-42.6.0.jar’ saved [1081604/1081604]



In [18]:
!mkdir jars
!mv postgresql-42.6.0.jar jars/

In [32]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, explode
import pyspark.sql.functions as f


In [33]:
# Extract
# spark = SparkSession.builder.appName("ETL Pipeline").getOrCreate()
spark = SparkSession.builder \
    .appName("Write to PostgreSQL") \
    .config("spark.executor.extraClassPath", "/content/jars/postgresql-<version>.jar") \
    .config("spark.driver.extraClassPath", "/content/jars/postgresql-<version>.jar") \
    .getOrCreate()


In [34]:
df = spark.read.text('WordData.txt')
df.show(truncate =False)

+-----------------------------------------------------------------------------------------+
|value                                                                                    |
+-----------------------------------------------------------------------------------------+
|This is a Japanese doll                                                                  |
|The team members were hard to tell apart since they all wore their hair in a ponytail    |
|As the years pass by we all know owners look more and more like their dogs               |
|If you don't like toenails you probably shouldn't look at your feet                      |
|He was disappointed when he found the beach to be so sandy and the sun so sunny          |
|When he encountered maize for the first time he thought it incredibly corny              |
|Situps are a terrible way to end your day                                                |
|Toddlers feeding raccoons surprised even the seasoned park ranger              

In [35]:
df2 = df.withColumn("splitedData", f.split("value"," "))
df3 = df2.withColumn('splitedData', explode("splitedData"))
wordsDF = df3.groupBy('splitedData').count()
wordsDF.show()

+-----------+-----+
|splitedData|count|
+-----------+-----+
|   Tomorrow|    4|
|         If|    8|
|      leave|    4|
|      corny|    4|
|        day|    4|
|preoccupied|    4|
|       even|    8|
|      crazy|    4|
|    bananas|    4|
|     priest|    4|
|        did|    4|
|    whether|    4|
|     Having|    4|
|        I'm|    4|
|      crime|    4|
|  surprised|    4|
|      James|    4|
|      could|    8|
|        buy|    4|
|        him|    8|
+-----------+-----+
only showing top 20 rows



In [36]:
# Load
driver = "org.postgresql.Driver"
url = "jdbc:postgresql://db-postgres.cnc1f54atosg.us-east-2.rds.amazonaws.com/"
table="pyspark_schema.wordCount"
user = "postgres"
password = "password"

In [37]:
jdbc_url = "jdbc:postgresql://db-postgres.cnc1f54atosg.us-east-2.rds.amazonaws.com:5432/postgres"
connection_properties = {
    "user": "postgres",
    "password": "password",
    "driver": "org.postgresql.Driver"
}

In [39]:
# wordsDF.write \
#     .format("jdbc") \
#     .option("url", jdbc_url) \
#     .option("dbtable", "dummy_table") \
#     .mode("overwrite") \
#     .options(**connection_properties) \
#     .save()